In [1]:
# Import the libraries
import os
import pandas as pd
import geopandas as gpd
import sqlalchemy as sql
from geoalchemy2 import Geometry, WKTElement

In [2]:
# Load the environment variables and constants
# Using postgres database to store the data
DATBASE_HOST = os.getenv('ENVISO_INSIGHT_DATABASE_HOSTNAME')
DATBASE_NAME = 'spatial_db_project'
DATBASE_USER = os.getenv('ENVISO_INSIGHT_DATABASE_USERNAME')
DATBASE_PASS = os.getenv('ENVISO_INSIGHT_DATABASE_PASSWORD')

In [3]:
# Load the house price data
df_house_price = pd.read_csv('dataset/house_price_data.csv')
df_house_price.head()
df_house_price.shape

(34348, 23)

In [4]:
# Check for NULL values within the dataset
df_house_price.isna().sum()

id                  0
id_string           0
city                0
locality          208
project         14169
dev_name        16999
furnishing         10
area              776
bathroom_num       14
bedroom_num         0
floor_count      2860
floor_num        2781
latitude            0
longitude           0
post_date           0
poster_name         1
price               0
title              10
trans              30
type                0
url                10
user_type           0
desc               10
dtype: int64

In [5]:
# To keep things simple, we will start by removing the fields that have NULL values
# This will make our life easier
# Prepare a list of columns to be dropped
list_columns_dropped = ['id_string','locality','project','dev_name','furnishing','area','bathroom_num','floor_count','floor_num','poster_name','title','trans','url','desc']
df_house_price = df_house_price.drop(labels = list_columns_dropped, axis = 1)
df_house_price.head()

,id,city,bedroom_num,latitude,longitude,post_date,price,type,user_type
0,45349857,Mumbai,1,19.201336,72.825882,11/01/20,9000,Apartment,Agent
1,45960973,Mumbai,1,19.410704,72.833592,11/01/20,8060,Apartment,Agent
2,46688849,Mumbai,1,19.467032,72.801612,13/12/19,8000,Apartment,Agent
3,44696119,Mumbai,1,19.407257,72.836006,13/01/20,8000,Apartment,Agent
4,46742851,Mumbai,1,19.360215,72.850167,17/12/19,9000,Apartment,Owner


In [6]:
# Check the datatype of the column post_date
df_house_price.dtypes

id               int64
city            object
bedroom_num      int64
latitude       float64
longitude      float64
post_date       object
price            int64
type            object
user_type       object
dtype: object

In [7]:
# Convert the post_date datatype from object to timestamp
# object datatype in pandas is string in basic python
df_house_price['post_date'] = pd.to_datetime(df_house_price['post_date'])
df_house_price.head()

,id,city,bedroom_num,latitude,longitude,post_date,price,type,user_type
0,45349857,Mumbai,1,19.201336,72.825882,2020-11-01,9000,Apartment,Agent
1,45960973,Mumbai,1,19.410704,72.833592,2020-11-01,8060,Apartment,Agent
2,46688849,Mumbai,1,19.467032,72.801612,2019-12-13,8000,Apartment,Agent
3,44696119,Mumbai,1,19.407257,72.836006,2020-01-13,8000,Apartment,Agent
4,46742851,Mumbai,1,19.360215,72.850167,2019-12-17,9000,Apartment,Owner


In [8]:
# Convert the pandas dataframe to a geopandas dataframe
# This will make a geometry column based on the latitude and longitude
gdf_house_price = gpd.GeoDataFrame(df_house_price, geometry = gpd.points_from_xy(df_house_price.longitude, df_house_price.latitude),  crs="EPSG:4326")
gdf_house_price.head()

,id,city,bedroom_num,latitude,longitude,post_date,price,type,user_type,geometry
0,45349857,Mumbai,1,19.201336,72.825882,2020-11-01,9000,Apartment,Agent,POINT (72.82588 19.20134)
1,45960973,Mumbai,1,19.410704,72.833592,2020-11-01,8060,Apartment,Agent,POINT (72.83359 19.41070)
2,46688849,Mumbai,1,19.467032,72.801612,2019-12-13,8000,Apartment,Agent,POINT (72.80161 19.46703)
3,44696119,Mumbai,1,19.407257,72.836006,2020-01-13,8000,Apartment,Agent,POINT (72.83601 19.40726)
4,46742851,Mumbai,1,19.360215,72.850167,2019-12-17,9000,Apartment,Owner,POINT (72.85017 19.36022)


In [9]:
# Insert the raw data into postgres table
# Create the database engine first
connection_string = f'postgresql://{DATBASE_USER}:{DATBASE_PASS}@{DATBASE_HOST}:5432/{DATBASE_NAME}'
db_engine = sql.create_engine(connection_string)

# Insert the data into a table in postgres
gdf_house_price.to_postgis(
    name = 'house_price_raw',
    con = db_engine,
    if_exists = 'replace',
    index = False
)